In [98]:
import tensorflow

In [99]:
tensorflow.__version__

'2.3.1'

In [100]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [101]:
data = pd.read_csv("D:/Data Science/Dataset/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [102]:
X = data.iloc[:,3:13]
y = data.iloc[:,13]

In [103]:
print(X.shape)
print(y.shape)

(10000, 10)
(10000,)


In [104]:
geography= pd.get_dummies(X['Geography'], drop_first = True)

In [105]:
gender= pd.get_dummies(X['Gender'], drop_first = True)

In [106]:
X= pd.concat([X, geography, gender], axis = 1)

In [107]:
X = X.drop(['Geography', 'Gender'], axis = 1)

In [108]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [109]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units = hp.Int('unit_'+str(i), min_value = 4, max_value =40, step =4), activation = 'relu'))
    model.add(layers.Dense(units = 1, activation = 'sigmoid'))
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2, 1e-3, 1e-4])),
                 loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [110]:
tuner = RandomSearch(build_model, objective = 'val_accuracy',
                    max_trials =5,
                    executions_per_trial =3,
                    directory = 'Trial2',
                    project_name = 'Churn_modelling')

In [111]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 40, 'step': 4, 'sampling': None}
unit_1 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 40, 'step': 4, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)

In [113]:
tuner.search(X_train, y_train, epochs =5, validation_data =(X_test, y_test) )

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.7933333516120911

Best val_accuracy So Far: 0.7938889066378275
Total elapsed time: 00h 01m 42s
INFO:tensorflow:Oracle triggered exit


In [114]:
tuner.results_summary()

Results summary
Results in Trial2\Churn_modelling
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 9
unit_0: 12
unit_1: 36
learning_rate: 0.001
unit_2: 8
unit_3: 12
unit_4: 32
unit_5: 8
unit_6: 8
unit_7: 32
unit_8: 36
Score: 0.7938889066378275
Trial summary
Hyperparameters:
num_layers: 9
unit_0: 40
unit_1: 28
learning_rate: 0.001
unit_2: 4
unit_3: 4
unit_4: 4
unit_5: 4
unit_6: 4
unit_7: 4
unit_8: 4
Score: 0.7933333516120911
Trial summary
Hyperparameters:
num_layers: 11
unit_0: 8
unit_1: 20
learning_rate: 0.001
unit_2: 32
unit_3: 12
unit_4: 8
unit_5: 12
unit_6: 36
unit_7: 36
unit_8: 24
unit_9: 4
unit_10: 4
Score: 0.7933333516120911
Trial summary
Hyperparameters:
num_layers: 6
unit_0: 24
unit_1: 36
learning_rate: 0.001
unit_2: 16
unit_3: 16
unit_4: 12
unit_5: 40
unit_6: 12
unit_7: 32
unit_8: 36
Score: 0.7888888915379842
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 12
unit_1: 8
learning_rate: 0.001
Score: 0.781888882